In [1]:
import pandas as pd
import numpy as np
import plotly.express as px

# simplest simulation: markov between 2 leagues

In [2]:
# initial state
n_league1 = 100
n_league2 = 0

In [3]:
# parameters of league change
p11 = 0.8
p22 = 0.6
ptrans = [[p11,1-p11],[1-p22,p22]]


In [20]:
nstep = 400
# start simulation
n1,n2 = n_league1,n_league2
simudata = []
simudata.append({
        'time_step':0,
        'n1':n1,
        'n2':n2

})
for kstep in range(nstep):
    n_11 = sum(np.random.rand(n1)<p11)
    n_12 = n1-n_11
    
    n_22 = sum(np.random.rand(n2)<p22)
    n_21 = n2-n_22
    
    n1 = n_11 + n_21
    n2 = n_22 + n_12
    
    simudata.append( {
        'time_step':kstep,
        'n1':n1,
        'n2':n2
    })
simudata = pd.DataFrame(simudata)

In [21]:
simudata

,time_step,n1,n2
0,0,100,0
1,0,78,22
2,1,67,33
3,2,69,31
4,3,67,33
...,...,...,...
396,395,68,32
397,396,59,41
398,397,69,31
399,398,64,36


In [30]:
px.line(simudata,x='time_step',y=['n1','n2'],labels={'variable':'number of DAO in each league'})

An initial big change (many league 1 converts to league 2), then stablize.

# How does the league transition impacts NEAR funding?

In [31]:
# initial variable
n1 = 100
n2 = 0
total_nearfund=0

In [32]:
# parameters 
p11 = 0.8
p22 = 0.6
near_fund_eachstep = [1,5] # how much fund is granted for each dao depending on their league. More fundings for league 2

In [36]:
nstep = 40

In [37]:
# start simulation
simudata = []
simudata.append({
        'time_step':0,
        'n1':n1,
        'n2':n2,
        'step_nearfund_granted':near_fund_eachstep[0]*n1 + near_fund_eachstep[1]*n2,
        'total_nearfund_granted':total_nearfund + near_fund_eachstep[0]*n1 + near_fund_eachstep[1]*n2

})
for kstep in range(nstep):
    n_11 = sum(np.random.rand(n1)<p11)
    n_12 = n1-n_11
    
    n_22 = sum(np.random.rand(n2)<p22)
    n_21 = n2-n_22
    
    n1 = n_11 + n_21
    n2 = n_22 + n_12
    
    nearfund_step = near_fund_eachstep[0]*n1 + near_fund_eachstep[1]*n2
    total_nearfund = total_nearfund + nearfund_step
    simudata.append( {
        'time_step':kstep,
        'n1':n1,
        'n2':n2,
        'step_nearfund_granted':nearfund_step,        
        'total_nearfund_granted':total_nearfund
    })
    
simudata = pd.DataFrame(simudata)

In [40]:
px.line(simudata,x='time_step',y=['step_nearfund_granted'])

No surprise, the near fund granted at each step is around a given range since the number of leagues is fluctuating around a certain number

TODO: 1) introduce natural growth of league 1? 2) add the impact backflow to the system. add that system revenue as a parameter too